In [1]:
import itertools
import random
import numpy as np
import pandas as pd
import math
import time
import simpy
import json
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
import matplotlib.pyplot as plt
import tkinter as tk
from PIL import ImageTk
import random
from datetime import datetime, timedelta
import os
from collections import defaultdict

In [2]:
######################
RFID_GATE_LINES = 2
PAPER_GATE_LINES = 2

PAPER_EMPS_PER_LINE = 1
RFID_EMPS_PER_LINE = 0.5
######################
RFID_SELECTION_RATE = 0.7

RFID_SCAN_TIME_MIN = 5
RFID_SCAN_TIME_MAX = 12
PAPER_SCAN_TIME_MIN = 10
PAPER_SCAN_TIME_MAX = 17

JOIN_RATE_HIGH_MEAN = 2
JOIN_RATE_HIGH_STD = 0.05

JOIN_RATE_AVG_MEAN = 8
JOIN_RATE_AVG_STD = 1

JOIN_RATE_LOW_MEAN = 60
JOIN_RATE_LOW_STD = 10

ERROR_RATE_RFID = 0.01
ERROR_RATE_PAPER = 0.035
# Creating the 'output' directory if it doesn't exist
os.makedirs('output', exist_ok=True)

In [3]:
arrivals = defaultdict(lambda: 0)
seller_waits = defaultdict(list)
event_log = []

def get_current_time(elapsed_seconds):
    start_time = datetime.strptime('06:30:00', '%H:%M:%S')
    current_time = start_time + timedelta(seconds=elapsed_seconds)
    formatted_current_time = current_time.strftime('%H:%M:%S')
    return formatted_current_time


# def avg_wait(raw_waits):
#     waits = [w for i in raw_waits.values() for w in i]
#     return round(np.mean(waits), 1) if waits else 0

def register_individual_arrival(time, person_id):
    arrivals[int(time)] += 1
    event_log.append({"event": "INDIVIDUAL_ARRIVAL", "time": get_current_time(time), "personId": person_id})

def queueing_to_scanner(person, card_type, gate_line, traffic_status, queue_begin, queue_end, scan_begin, scan_end, error_appearance, correction_begin, error_correction_time, correction_end):
    wait = queue_end - queue_begin
    seller_waits[int(queue_end)].append(wait)
    event_log.append({"event": "WAIT_IN_GATE_LINE", "person": f"id_{person}", "selected line": f"{card_type}_{gate_line}", "traffic status": traffic_status, "begin time": get_current_time(queue_begin), "end time": get_current_time(queue_end), "duration": round(wait, 2)})
    event_log.append({"event": "SCAN TICKET", "person": f"id_{person}", "selected line": f"{card_type}_{gate_line}", "traffic status": traffic_status, "begin time": get_current_time(scan_begin), "end time": get_current_time(scan_end), "duration": round(scan_end - scan_begin, 2)})
    if error_appearance:
        event_log.append({"event": "ERROR OCCURENCE AND CORRECTION", "person": f"id_{person}", "selected line": f"{card_type}_{gate_line}", "traffic status": traffic_status, "begin time": get_current_time(correction_begin), "end time": get_current_time(correction_end), "duration": round(error_correction_time, 2)})

def pick_shortest(lines):
    shuffled = list(zip(range(len(lines)), lines))
    random.shuffle(shuffled)
    shortest = shuffled[0][0]
    for i, line in shuffled:
        if len(line.queue) < len(lines[shortest].queue):
            shortest = i
            break
    return lines[shortest], shortest + 1

def generate_scan_time(card_type):
    if card_type == 'RFID':
        return random.uniform(RFID_SCAN_TIME_MIN, RFID_SCAN_TIME_MAX)
    elif card_type == 'paper':
        return random.uniform(PAPER_SCAN_TIME_MIN, PAPER_SCAN_TIME_MAX)
    
def check_traffic_status(env):
    current_time = get_current_time(env.now)

    if current_time < "08:30:00":
        return "low"
    elif "08:30:00" <= current_time < "09:30:00":
        return "avg"
    elif "09:30:00" <= current_time < "10:30:00":
        return "high"
    elif "10:30:00" <= current_time < "11:30:00":
        return "avg"
    elif "11:30:00" <= current_time < "12:30:00":
        return "high"
    elif "12:30:00" <= current_time < "14:30:00":
        return "low"
    elif "14:30:00" <= current_time < "15:30:00":
        return "avg"
    elif "15:30:00" <= current_time < "16:30:00":
        return "high"
    elif "16:30:00" <= current_time < "18:30:00":
        return "avg"
    elif "18:30:00" <= current_time < "19:30:00":
        return "high"
    else:
        return "low"

    
def generate_arrival_time(env):
    traffic_status = check_traffic_status(env)
    if traffic_status == 'high':
        arrival_time = max(0, random.normalvariate(JOIN_RATE_HIGH_MEAN, JOIN_RATE_HIGH_MEAN))
    elif traffic_status == 'low':
        arrival_time = max(0, random.normalvariate(JOIN_RATE_LOW_MEAN, JOIN_RATE_LOW_STD))
    else: # NORMAL
        arrival_time = max(0, random.normalvariate(JOIN_RATE_AVG_MEAN, JOIN_RATE_AVG_STD))        
    return arrival_time, traffic_status


def is_error(card_type):
    if card_type == 'RFID':
        return random.random() <= ERROR_RATE_RFID
    elif card_type == 'paper':
        return random.random() <= ERROR_RATE_PAPER
    
def generate_error_correction_time(card_type):
    if card_type == 'RFID':
        if RFID_EMPS_PER_LINE == 1:
            ERROR_CORRECTION_TIME = max(0, random.normalvariate(15, 5))
        elif 0.5 < RFID_EMPS_PER_LINE < 1:
            ERROR_CORRECTION_TIME = max(0, random.normalvariate(20, 5))
        else:
            ERROR_CORRECTION_TIME = max(0, random.normalvariate(30, 5))
    elif card_type == 'paper':
        ERROR_CORRECTION_TIME = max(0, random.normalvariate(10, 2))
        
    return ERROR_CORRECTION_TIME


def individual_arrival(env, rfid_gate_lines, paper_gate_lines):
    next_person_id = 0
    while True:
        card_type = random.choices(['RFID', 'paper'], weights=[RFID_SELECTION_RATE, 1 - RFID_SELECTION_RATE])[0]
        if card_type == 'RFID':
            gate_lines = rfid_gate_lines
        else:
            gate_lines = paper_gate_lines
        next_arrival, traffic_status = generate_arrival_time(env)
        yield env.timeout(next_arrival)

        env.process(purchasing_individual(env, next_person_id, gate_lines, card_type, traffic_status))
        next_person_id += 1

def purchasing_individual(env, person_id, gate_lines, card_type, traffic_status):
    queue_begin = env.now
    gate_line = pick_shortest(gate_lines)
    with gate_line[0].request() as req:
        yield req
        queue_end = env.now

        # SCANNING
        scan_begin = env.now
        scan_time = generate_scan_time(card_type=card_type)
        yield env.timeout(scan_time)
        scan_end = env.now
        
        
        # ERROR
        correction_begin = env.now
        error_appearance = is_error(card_type)
        if error_appearance:
            error_correction_time = generate_error_correction_time(card_type)
        else:
            error_correction_time = 0
        
        yield env.timeout(error_correction_time)
        correction_end = env.now
            
        queueing_to_scanner(person_id, card_type, gate_line[1], traffic_status, queue_begin, queue_end, scan_begin, scan_end, error_appearance, correction_begin, error_correction_time, correction_end)

env = simpy.Environment()

rfid_gate_lines = [simpy.Resource(env, capacity=RFID_EMPS_PER_LINE) for _ in range(RFID_GATE_LINES)]
paper_gate_lines = [simpy.Resource(env, capacity=PAPER_EMPS_PER_LINE) for _ in range(PAPER_GATE_LINES)]
all_gate_lines = [rfid_gate_lines, paper_gate_lines]


env.process(individual_arrival(env, rfid_gate_lines, paper_gate_lines))
env.run(until=14*60*60)


# Writing data to a JSON file
with open('output/events.json', 'w') as outfile:
    # input_string = f"""RFID Gates: {RFID_GATE_LINES} | Paper Gates: {PAPER_GATE_LINES} || Paper Employees per Line: {PAPER_EMPS_PER_LINE} | RFID Employees per Line: {RFID_EMPS_PER_LINE}"""
    # config_string = f"""RFID Selection Rate: {RFID_SELECTION_RATE} || RFID Scan Time (Min): {RFID_SCAN_TIME_MIN}| RFID Scan Time (Max): {RFID_SCAN_TIME_MAX} || Paper Scan Time (Min): {PAPER_SCAN_TIME_MIN}| Paper Scan Time (Max): {PAPER_SCAN_TIME_MAX} || Join Rate High Mean: {JOIN_RATE_HIGH_MEAN}| Join Rate High Std: {JOIN_RATE_HIGH_STD} || Join Rate Avg Mean: {JOIN_RATE_AVG_MEAN}| Join Rate Avg Std: {JOIN_RATE_AVG_STD} || Join Rate Low Mean: {JOIN_RATE_LOW_MEAN}| Join Rate Low Std: {JOIN_RATE_LOW_STD} || Error Rate RFID: {ERROR_RATE_RFID}| Error Rate Paper: {ERROR_RATE_PAPER}"""
    json.dump({"RFID GATES": RFID_GATE_LINES, 
               "PAPER GATES": PAPER_GATE_LINES,
               "RFID EMPLOYEES": int(RFID_GATE_LINES * RFID_EMPS_PER_LINE),
               "PAPER EMPLOYEES": PAPER_GATE_LINES * PAPER_EMPS_PER_LINE,
               "events": event_log}, outfile, indent=4)

# Phân tích thời gian chờ

In [4]:
with open('output/events.json', 'r') as f:
    data = json.load(f)
output_df = pd.DataFrame(data)
events_df = pd.json_normalize(output_df['events'])
output_df = pd.concat([output_df.drop('events', axis=1), events_df], axis=1)
output_df['gate_type'] = output_df['selected line'].str[:-2]
output_df.sort_values('person')


,RFID GATES,PAPER GATES,RFID EMPLOYEES,PAPER EMPLOYEES,event,person,selected line,traffic status,begin time,end time,duration,gate_type
0,2,2,2,2,WAIT_IN_GATE_LINE,id_0,paper_2,low,06:31:01,06:31:01,0.00,paper
1,2,2,2,2,SCAN TICKET,id_0,paper_2,low,06:31:01,06:31:17,16.17,paper
2,2,2,2,2,WAIT_IN_GATE_LINE,id_1,RFID_2,low,06:31:57,06:31:57,0.00,RFID
3,2,2,2,2,SCAN TICKET,id_1,RFID_2,low,06:31:57,06:32:07,10.65,RFID
21,2,2,2,2,SCAN TICKET,id_10,paper_2,low,06:41:14,06:41:28,14.01,paper
...,...,...,...,...,...,...,...,...,...,...,...,...
2103,2,2,2,2,WAIT_IN_GATE_LINE,id_997,RFID_2,high,09:45:53,09:52:08,374.23,RFID
2120,2,2,2,2,SCAN TICKET,id_998,RFID_1,high,09:52:30,09:52:38,8.30,RFID
2119,2,2,2,2,WAIT_IN_GATE_LINE,id_998,RFID_1,high,09:45:54,09:52:30,395.82,RFID
2112,2,2,2,2,SCAN TICKET,id_999,RFID_2,high,09:52:16,09:52:25,9.29,RFID


In [5]:
wait_time_df = output_df.groupby(['RFID GATES', 'PAPER GATES', 'RFID EMPLOYEES', 'PAPER EMPLOYEES', 'person', 'traffic status', 'gate_type'])['duration'].sum().reset_index()
wait_time_df = wait_time_df.rename(columns={'duration': 'wait time'})
wait_time_df.to_csv('output/output.csv', index=False)
wait_time_df

,RFID GATES,PAPER GATES,RFID EMPLOYEES,PAPER EMPLOYEES,person,traffic status,gate_type,wait time
0,2,2,2,2,id_0,low,paper,16.17
1,2,2,2,2,id_1,low,RFID,10.65
2,2,2,2,2,id_10,low,paper,14.01
3,2,2,2,2,id_100,low,paper,15.02
4,2,2,2,2,id_1000,high,paper,66.10
...,...,...,...,...,...,...,...,...
9125,2,2,2,2,id_995,high,RFID,398.79
9126,2,2,2,2,id_996,high,RFID,374.22
9127,2,2,2,2,id_997,high,RFID,382.28
9128,2,2,2,2,id_998,high,RFID,404.12


In [6]:
wait_time_df.groupby(['RFID GATES', 'PAPER GATES', 'RFID EMPLOYEES', 'PAPER EMPLOYEES', 'gate_type'])['wait time'].mean().reset_index()

,RFID GATES,PAPER GATES,RFID EMPLOYEES,PAPER EMPLOYEES,gate_type,wait time
0,2,2,2,2,RFID,584.255338
1,2,2,2,2,paper,38.411822


In [7]:
wait_time_df.groupby(['RFID GATES', 'PAPER GATES', 'RFID EMPLOYEES', 'PAPER EMPLOYEES', 'traffic status'])['wait time'].mean().reset_index()

,RFID GATES,PAPER GATES,RFID EMPLOYEES,PAPER EMPLOYEES,traffic status,wait time
0,2,2,2,2,avg,141.341466
1,2,2,2,2,high,534.421832
2,2,2,2,2,low,109.527667
